Open in [nbviewer](https://nbviewer.jupyter.org/github/luiarthur/ucsc_litreview/blob/master/notes/Litreview.ipynb)

# [Bayesian inference for intratumour heterogeneity in mutations and copy number variation](../pdf/bayesTumor.pdf)

## Glossary

somatic
: relating to the body